## Re-examine SF_Uncert metric in DDF regarding season length (SF batch)

In [1]:
# development code
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import glob
import os, sys

In [3]:
# add the path the scripts
sys.path.insert(0, "../scripts/")

In [4]:
# import rubin_sim python modules
import rubin_sim.maf.db as db
import rubin_sim.maf.metrics as metrics
import rubin_sim.maf.slicers as slicers
import rubin_sim.maf.stackers as stackers
import rubin_sim.maf.plots as plots
import rubin_sim.maf.metricBundles as metricBundles
import rubin_sim.maf as maf
from rubin_sim.maf.utils import m52snr

# footprint utils
from rubin_sim.scheduler.utils import footprints

# import convenience functions
from opsimUtils import *

# import custom stacker/metrics from script
from agnstructure import *

# print version
import rubin_sim

rubin_sim.__version__

'0.10.1.dev64+gc32f68b'

### 1. Pull rolling opsims to evaluate 

In [5]:
families = maf.archive.get_family_descriptions()
family_list = families.index.values

# pull roll runs
fam = ["ddf season length"]
baseline_run = families.loc["ddf season length", "reference"]
ddf_runs = np.concatenate([[baseline_run], families.explode("run").loc[fam, "run"]])
ddf_runs

array(['baseline_v2.1_10yrs', 'ddf_season_length_slf0.10_v2.1_10yrs',
       'ddf_season_length_slf0.15_v2.1_10yrs',
       'ddf_season_length_slf0.20_v2.1_10yrs',
       'ddf_season_length_slf0.25_v2.1_10yrs',
       'ddf_season_length_slf0.30_v2.1_10yrs',
       'ddf_season_length_slf0.35_v2.1_10yrs'], dtype=object)

### 2. Test SF Uncert on the baseline

In [6]:
dbDir = rubin_sim.data.get_data_dir()
outDir = "/home/jovyan/mount/ResearchData/MAFOutput/ddf_sf/ResultsDb"
metricDataPath = "/home/jovyan/mount/ResearchData/MAFOutput/ddf_sf/MetricData"

if not os.path.exists(os.path.abspath(outDir)):
    os.makedirs(os.path.abspath(outDir))

if not os.path.exists(os.path.abspath(metricDataPath)):
    os.makedirs(os.path.abspath(metricDataPath))

In [7]:
# create dict for storing dbs
opSimDbs, resultDbs = connect_dbs(dbDir, outDir, dbRuns=ddf_runs)

In [8]:
ddfFields = ['COSMOS', 'XMM-LSS', 'ELAISS1', 'ECDFS', 'EDFS']
agn_m5 = {"u": 22.89, "g": 23.94, "r": 23.5, "i": 22.93, "z": 22.28, "y": 21.5}

#### 2.1 Default 10 bins

In [9]:
# metric name template
metricNameTemp1 = 'sf_{}_{}_20'

# custom sf bins
nbin = 20
mybins = np.logspace(0, np.log10(3650), nbin+1)
myweights = np.full(nbin, 1/nbin)
summaryMetrics = [metrics.MedianMetric()]

for run in ddf_runs:
    print(f'Runing metrics on Opsim: {run}!')
    print('*****************************************')
    bdict1 = {}

    for ddf in ddfFields:
        # custom slicer
        nside = 128
        radius = 1.8
        hp_mask = ddf_hp_mask(ddf, radius, nside)
        slicer = slicers.HealpixSubsetSlicer(nside, np.where(hp_mask == 1)[0])
        slicer.slicerName = 'HealpixSlicer'  # hack maf
    
        for band in 'ugrizy':
            constraint = f'filter = "{band}"'
            # sf uncert
            sfuncert = SFUncertMetric(mag=agn_m5[band], 
                                      bins=mybins, weight=myweights, snr_cut=0, 
                                      metricName=metricNameTemp1.format(ddf, band))
            bdict1[sfuncert.name] = metricBundles.MetricBundle(sfuncert, slicer,
                                                               constraint)
            # median m5
            m5p50 = metrics.MedianMetric('fiveSigmaDepth', 
                                        metricName = f'm5Median_{ddf}_{band}')         
            m5p50mb = metricBundles.MetricBundle(m5p50, slicer, constraint)
            bdict1[m5p50.name] = m5p50mb
            
            # m5 sigma
            m5rms = metrics.RobustRmsMetric('fiveSigmaDepth', 
                                             metricName = f'm5rms_{ddf}_{band}')
            m5rmsmb = metricBundles.MetricBundle(m5rms, slicer, constraint)
            bdict1[m5rms.name] = m5rmsmb
            
    for key in bdict1:
        bdict1[key].setRunName(run)
        bdict1[key].setSummaryMetrics(summaryMetrics)

    gp = metricBundles.MetricBundleGroup(bdict1, opSimDbs[run], outDir=metricDataPath,
                                         resultsDb=resultDbs[run], verbose=False)
    gp.runAll()

Runing metrics on Opsim: baseline_v2.1_10yrs!
*****************************************
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes


/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should p

Runing metrics on Opsim: ddf_season_length_slf0.10_v2.1_10yrs!
*****************************************
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes


/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should p

Runing metrics on Opsim: ddf_season_length_slf0.15_v2.1_10yrs!
*****************************************
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes


...

/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should p

Runing metrics on Opsim: ddf_season_length_slf0.30_v2.1_10yrs!
*****************************************
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes


/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should p

Runing metrics on Opsim: ddf_season_length_slf0.35_v2.1_10yrs!
*****************************************
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes
Healpix slicer using NSIDE=128, approximate resolution 27.483891 arcminutes


/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should probably set useCache=False in slicer.
  warnings.warn(
/opt/conda/envs/rubin/lib/python3.10/site-packages/rubin_sim/maf/slicers/baseSpatialSlicer.py:118: UserWarning: Warning:  Loading maps but cache on.Should p